In [7]:
import requests
import time
import os
import pandas as pd
import numpy as np
import random
import re

# !pip3 install -U spacy
import spacy
from spacy import displacy
from time import sleep
from os import path
from pandas import DataFrame
from bs4 import BeautifulSoup
from random import randint

In [2]:
def get_contents(soup, content_text):
  try:
    parents_blacklist=['[document]','html','head',
                       'style','script','body',
                       'section','tr',
                       'td','label','ul','header',
                       'aside']
    content=''
    text=soup.find_all(text=True)

    
    for t in text:
        if t.parent.name not in parents_blacklist and len(t) > 5:
            content=content+t+' '
    content_text.append(content)
  except Exception:
    content_text.append('')
    pass

In [52]:
def preprocessor_final(text):
    if isinstance((text), (str)):
        text = re.sub('<[^>]*>', ' ', text)
#         text = re.sub('[\W_^\$]+', ' ', text.lower())
        return text
    if isinstance((text), (list)):
        return_list = []
        for i in range(len(text)):
            temp_text = re.sub('<[^>]*>', ' ', text[i])
#             text = re.sub('[\W_^\$]+', ' ', text.lower())
            return_list.append(temp_text)
        return(return_list)
    else:
        pass

In [84]:
def convert_to_ner_data_craig(df):
    data = list()
    simple_data = list()
    for col in df.columns:
        if col == 'Address': 
            new_name = 'address'
        elif col == "Beds":
            new_name = 'bed_range'
        elif col == "Rent":
            new_name = 'price_range'
        else:
            new_name = col
        if col != 'Unnamed: 0' and col != 'Link':
            simple_data = simple_data + df[col].apply(lambda x: (x, new_name)).to_list()
    return simple_data

In [83]:
def convert_to_ner_data_thiya(df):
    data = list()
    simple_data = list()
    for col in df.columns:
        if col == 'Company': 
            new_name = 'owner'
        else:
            new_name = col
        if col != 'Unnamed: 0' and col != 'Name':
            simple_data = simple_data + df[col].apply(lambda x: (x, new_name)).to_list()
    return simple_data

In [19]:
def display_entities(model_path, content):
    nlp = spacy.load(model_path)
    doc = nlp(content)
    displacy.render(doc, style="ent", jupyter = True)
    

In [105]:
def quantitative_results(model_path, test_data):
    nlp = spacy.load(model_path)
    correct = 0
    total = 0
    results_array = []
    for element in test_data:
        if isinstance(element[0], str):
            text = " ".join(element[0].split())
        else:
            continue
        doc = nlp(text)
        for word in doc.ents:
            if word.label_ == element[1]:
                total = total + 1
                correct = correct + 1
    #             print("Correct", total, correct, word.text, word.label_, element[1])
                results_array.append([word.text, element[1], word.label_])
            else:
                total = total + 1
    #             print("Incorrect", total, correct, word.text, word.label_, element[1])
                results_array.append([word.text, element[1], word.label_])

    print("Corret, Total, Accuracy: ", correct, total, (correct/total) * 100)
    
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(pd.DataFrame(results_array, columns = ['Text', 'Ground Truth', 'Prediction']))

In [61]:
page_2015 ="https://web.archive.org/web/20150217144133/https://www.apartments.com/chicago-il/"
page_2022 = 'https://web.archive.org/web/20220819053652/https://www.apartments.com/chicago-il/'

page=requests.get(page_2015)
soup=BeautifulSoup(page.text,'html.parser')
content_text = []
get_contents(soup, content_text)
content = "".join(content_text)

In [56]:
clean_content = preprocessor_final(content)
clean_content = ' '.join(clean_content.split())

In [57]:
display_entities("en_core_web_lg", clean_content)

In [63]:
display_entities("models/model-best-all-fields", clean_content)

In [76]:
display_entities("models/model-best-all-fields/", "Studio - 3 Br")

In [78]:
display_entities("models/model-best-all-fields/", "Lakeview Aparments")

In [89]:
display_entities("models/model-best-all-fields/", "Inspira Property Management")

/opt/anaconda3/envs/webscraper_venv/lib/python3.10/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [77]:
display_entities("models/model-best-all-fields/", "$1,452 - $4,413")

In [73]:
display_entities("models/model-best-all-fields/", "6054 Drexel Avenue, Chicago, IL 60637 Lakeview Apartments")

In [74]:
display_entities("models/model-best-all-fields/", "567-907-5673")

In [75]:
display_entities("models/model-best-all-fields/", "9078906543")
# This identifies that we need more data because the model can correctly identify the phone number with dashes but not a phone number with no dashes

In [88]:
info_craig = pd.read_csv("../training_data/apartments_com_prices_2015_2022.csv")
info_thiya = pd.read_csv("../training_data/pm.csv")

test_data_craig_simple = convert_to_ner_data_craig(info_craig)
test_data_thiya_simple = convert_to_ner_data_thiya(info_thiya)

test_data_craig_simple
test_data_thiya_simple

[('Aartie Aiyer Realty LLC', 'owner'),
 ('Above All Property Management', 'owner'),
 ('Accel Property Management & Investments', 'owner'),
 ('Ackley Florida Property Management', 'owner'),
 ('Acres Real Estate, LLC', 'owner'),
 ('Action Properties', 'owner'),
 ('ADEA Property Management Co.', 'owner'),
 ('Advantage Gold Realty Inc.', 'owner'),
 ('AIP Management, LLC', 'owner'),
 ('Alarca Realty', 'owner'),
 ('Alder Property Management', 'owner'),
 ('Alderson Properties', 'owner'),
 ('Alderson Properties', 'owner'),
 ('Alger Real Estate Group', 'owner'),
 ('Algood Hawaii, LLC', 'owner'),
 ('All County Metroplex', 'owner'),
 ('All County Polk Property Mgmt', 'owner'),
 ('Allred Properties CRMC®', 'owner'),
 ('Alpizar Real Estate, LLC', 'owner'),
 ('Alpizar Real Estate, LLC', 'owner'),
 ('Ameritex Property Management', 'owner'),
 ('AMOSO Properties', 'owner'),
 ('Anderson Realty', 'owner'),
 ('Aparicio Realty & Management, LLC', 'owner'),
 ('ARC Environmental, Inc.', 'owner'),
 ('ARG REAL

In [106]:
quantitative_results("models/model-best", test_data_craig_simple)

Corret, Total, Accuracy:  2022 2076 97.39884393063583
                                          Text Ground Truth   Prediction
0                              111 W Wacker Dr      address      address
1                                425 W Surf St      address      address
2                            445 E Illinois St      address      address
3                       5222-5238 S Drexel Ave      address      address
4                        5550 S DORCHESTER Ave      address      address
5                               900 S Clark St      address      address
6                        2727 N Pine Grove Ave      address      address
7                              860 N Dewitt Pl      address      address
8                       1164-1212 W Madison St      address      address
9                             555 W Madison St      address      address
10                          180 N Jefferson St      address      address
11                             161 W Kinzie St      address      addre

In [ ]:
quantitative_results("models/model-best", test_data_thiya_simple)

In [ ]:
quantitative_results("models/model-best-all-fields", test_data_craig_simple)

In [ ]:
quantitative_results("models/model-best-all-fields", test_data_thiya_simple)